# Mod 2 Summative Project - Soccer Match DB Transfer and Analysis

In [1]:
from SqlConn import SqlConn
from MongoHandler import MongoHandler
from PandaHandler import PandaHandler
from api_pull import api_pull
import time
import numpy as np
import json
import requests
import pandas as pd
import datetime
from matplotlib.image import imread
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
sql = SqlConn('database.sqlite')

Connection status: Active


In [10]:
df = sql.matches_df([2011])
df.sample(20)

Connection status: Active


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
295,1493,D1,2011,2011-12-10,Augsburg,M'gladbach,1,0,H
1193,1916,D1,2012,2013-05-11,Schalke 04,Stuttgart,1,2,A
539,1681,D2,2011,2012-04-11,Ein Frankfurt,Ingolstadt,1,1,D
885,1728,D1,2012,2012-12-01,Bayern Munich,Dortmund,1,1,D
936,1968,D1,2012,2013-01-18,Schalke 04,Hannover,5,4,H
724,1707,D1,2012,2012-09-29,Werder Bremen,Bayern Munich,0,2,A
687,2146,D2,2012,2012-09-21,Aalen,Erzgebirge Aue,2,0,H
806,2081,D2,2012,2012-11-04,Bochum,Cottbus,2,2,D
1183,2059,D1,2012,2013-05-05,Hamburg,Wolfsburg,1,1,D
71,1150,D2,2011,2011-08-22,St Pauli,Duisburg,2,1,H


In [11]:
sql.close_conn()

Closing connection
Connection status: Closed


In [12]:
df = PandaHandler.tot_home_goals_scored(df)
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,tot_home_goals
0,1133,D2,2011,2011-07-15,Cottbus,Dresden,2,1,H,46
1,1167,D2,2011,2011-07-15,Greuther Furth,Ein Frankfurt,2,3,A,56
2,1551,D2,2011,2011-07-15,Frankfurt FSV,Union Berlin,1,1,D,51
3,1550,D2,2011,2011-07-16,Erzgebirge Aue,Aachen,1,0,H,40
4,1678,D2,2011,2011-07-16,St Pauli,Ingolstadt,2,0,H,68


In [6]:
df = PandaHandler.win_loss_draw(df)
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,tot_home_goals,tot_home_win,tot_home_loss,tot_home_draw
0,1133,D2,2011,2011-07-15,Cottbus,Dresden,2,1,H,18,8,15,11
1,1167,D2,2011,2011-07-15,Greuther Furth,Ein Frankfurt,2,3,A,46,20,4,10
2,1551,D2,2011,2011-07-15,Frankfurt FSV,Union Berlin,1,1,D,20,7,13,14
3,1550,D2,2011,2011-07-16,Erzgebirge Aue,Aachen,1,0,H,19,8,15,11
4,1678,D2,2011,2011-07-16,St Pauli,Ingolstadt,2,0,H,36,18,8,8


In [7]:
df_goals_wins = df.groupby('HomeTeam')[['Season', 'tot_home_goals', 'tot_home_win', 'tot_home_loss', 'tot_home_draw']].first()
df_goals_wins.index.rename('Team', inplace=True)
df_goals_wins.columns = ['Season', 'GoalsScored', 'Wins', 'Losses', 'Draws']
df_goals_wins.head()

,Season,GoalsScored,Wins,Losses,Draws
Team,,,,,
Aachen,2011,15,6,15,13
Augsburg,2011,20,8,12,14
Bayern Munich,2011,49,23,7,4
Bochum,2011,23,10,17,7
Braunschweig,2011,21,10,9,15


In [ ]:
df_goals_wins['RainGames'] = 2
df_goals_wins['RainWins'] = 1
df_goals_wins['NonRainWins'] = 1
df_goals_wins['RainWin%'] = 0.54321
df_goals_wins['NonRainWin%'] = 0.54321
df_goals_wins['%ChangeWinWithRain'] = 0.54321
df_goals_wins.head()

In [ ]:
# RainGames    RainWins    NonRainWins    RainWin%    NonRainWin%    %ChangeWinWithRain

In [ ]:
season = str(int(df_goals_wins.iloc[0].Season))
cols = ['Wins', 'Losses', 'Draws']
bar_x = [season+'_Wins', season+'_Losses', season+'_Draws']

for i in range(len(df_goals_wins)):
    bar_y = list(df_goals_wins[cols].iloc[i])
    team_name = df_goals_wins.index[i]
    plt.bar(x=bar_x, height=bar_y)
    plt.title(team_name)
    plt.savefig('hist_images/{}.png'.format(team_name))
    plt.clf()

In [ ]:
df_goals_wins['graph'] = [imread('hist_images/{}.png'.format(team_name)).tolist() for team_name in df_goals_wins.index]
df_goals_wins.head()

### Mongo DB

Please ensure that your Mongo Database has been opened through the terminal for smooth operation.

In [ ]:
client = MongoHandler("mongodb://127.0.0.1:27017/")

In [ ]:
collection = client.make_collection('team_stats_db', 'team_stats_collection')
collection

In [ ]:
client.clear_collection('team_stats_db','team_stats_collection')

In [ ]:
results = collection.insert_many(MongoHandler.list_of_dicts(df_goals_wins))
results.inserted_ids

In [ ]:
query = client.query_db('team_stats_db', 'team_stats_collection')
for item in query:
    print(item)

# Working Space

In [ ]:
# The name of the team
# The total number of goals scored by the team during the 2011 season
# The total number of wins the team earned during the 2011 season
# A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
# The team's win percentage on days where it was raining during games in the 2011 season.

In [ ]:
# Query the SQL database
# Calculate summary statistics
# Get the weather data from the DarkSky API
# Load the data into MongoDB

In [ ]:
berlin_coordinates = 52.5200, 13.4050

In [ ]:
class MongoHandler():
    pass

In [ ]:
class WeatherGetter():
    pass

In [ ]:
from PIL import Image
from IPython.display import HTML

df_goals_wins['graph'] = ['<img src="hist_images/{}.png"/>'.format(team_name) for team_name in df_goals_wins.index]
HTML(df_goals_wins.head().to_html(escape=False))

In [ ]:
', '.join([2011, 2012])